In [9]:
def msg2():
    response = messagebox.showinfo('Information', 'The search has no result.\nKindly check.')
    customtkinter.CTkLabel(root, text=response)

def Linkedin_webscrap(search):
    from selenium import webdriver
    from selenium.webdriver.common.by import By
    from selenium.webdriver.common.keys import Keys
    from webdriver_manager.chrome import ChromeDriverManager
    from bs4 import BeautifulSoup
    import requests
    import pandas as pd
    import time
    import string
    import re
    import time
    from tqdm import tqdm, trange
    try:
        
        # email_id = input('Kindly provide email address for LinkedIn login:')

        # pswd = input('Password:')

        driver = webdriver.Chrome(ChromeDriverManager().install())

        driver.maximize_window()

        # Opening linkedIn's login page
        driver.get("https://linkedin.com/login")

        # waiting for the page to load
        time.sleep(3)

        # entering username
        username = driver.find_element(by=By.ID, value="username")

        # Enter Your Email Address
        # username.send_keys(email_id)
        username.send_keys("akumar1_mba21@thapar.edu")  

        # entering password
        pword = driver.find_element(by=By.ID, value="password")

        # Enter Your Password
        # pword.send_keys(pswd)
        pword.send_keys("qH3u!TqAMp9,8)J")

        # Clicking on the log in button
        # Format (syntax) of writing XPath --> 
        # //tagname[@attribute='value']
        driver.find_element(by=By.XPATH, value="//button[@type='submit']").click()
        # another XPATH "//*[@id="organic-div"]/form/div[3]/button"

        time.sleep(3)

        # Opening search term on linkedin
        url = 'https://www.linkedin.com/search/results/content/?keywords='+search

        driver.get(url) # this will open the link
        src = driver.page_source
        soup = BeautifulSoup(src, 'lxml')

        # Find the post blocks
        containers = soup.find_all('div', {'class': re.compile('^feed-shared-update-v2 feed-shared-update-v2--minimal-padding full-height relative.*')})
    
    except:
        msg2()

    
    
    # Infinite Scrolling
    time.sleep(3)

    start = time.time()

    # will be used in the while loop
    initialScroll = 0
    finalScroll = 10000

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        driver.execute_script(f"window.scrollTo({initialScroll}, {finalScroll})")
        last_position = driver.execute_script("return window.scrollY")
        # this command scrolls the window starting from
        # the pixel value stored in the initialScroll 
        # variable to the pixel value stored at the
        # finalScroll variable
        initialScroll = finalScroll
        finalScroll += 10000

        # we will stop the script for 3 seconds so that 
        # the data can load
        time.sleep(2)
        # You can change it as per your needs and internet speed

        end = time.time()

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")

        if last_position == last_height:
            break
        last_height = new_height

#     driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#     driver.execute_script("window.scrollTo(document.body.scrollHeight, 0);")

    time.sleep(2)

    src = driver.page_source
    soup = BeautifulSoup(src, 'lxml')

    # Find the post blocks
    containers = soup.find_all('div', {'class': re.compile('^feed-shared-update-v2 feed-shared-update-v2--minimal-padding full-height relative.*')})

    # Lists that we will iterate to
    post_account_names = []
    post_descs = []
    post_acc_links=[]
    post_links = []

    post_link_buttons = driver.find_elements(By.XPATH, "//button[@class='feed-shared-control-menu__trigger artdeco-button artdeco-button--tertiary artdeco-button--muted artdeco-button--1 artdeco-button--circle artdeco-dropdown__trigger artdeco-dropdown__trigger--placement-bottom ember-view']")

    for i in range(len(post_link_buttons)):
        driver.execute_script('arguments[0].scrollIntoView(true);', post_link_buttons[i])

        initialScroll = driver.execute_script("return window.scrollY")

        finalScroll = initialScroll - 216

        driver.execute_script(f"window.scrollTo({initialScroll}, {finalScroll})")
        post_link_buttons[i].click()
        time.sleep(2)

        # a = driver.find_elements(By.XPATH,"//*[contains(@class,'feed-shared-update-v2 feed-shared-update-v2--minimal-padding full-height relative')]")
        post_link_div = driver.find_element(By.XPATH,"//*[contains(@class,'option-share-via')]")
        # post_link_div
        post_link_div.click()

    time.sleep(2)
    src = driver.page_source
    soup = BeautifulSoup(src, 'lxml')

    post_link_click = soup.find_all('a', {'class': 'artdeco-toast-item__cta'})
    for i in range(len(post_link_click)):
        post_links.append(post_link_click[i]['href'])

    post_links.reverse()

    # Looping through the posts and appending them to the lists
    for container in containers:
    #     # Try function to make sure its a post and not a promotion
    #     try:

        # Posting Account name
        try:
            post_by = container.find('span', {'class': 'feed-shared-actor__title'}).get_text().strip().split("\n")[0]
            post_account_names.append(post_by)
        except:
            post_account_names.append("NA")

        # Post description
        try:
            post_desc_div = container.find('div', {'class': 'feed-shared-text relative feed-shared-update-v2__commentary'})
            post_descs.append(post_desc_div.get_text().strip())
        except:
            post_descs.append("NA")

        # Posting Account Link
        try:
            post_acc_link = container.find('a', {'class': 'app-aware-link feed-shared-actor__container-link relative display-flex flex-grow-1'})['href']
            post_acc_links.append(post_acc_link)
        except:
            post_acc_links.append("NA")


    data = {
        "Posting Account Name": post_account_names,
        "Post Description": post_descs,
        "Posting Account Link": post_acc_links,
        "Post Link": post_links
    }


    df = pd.DataFrame(data)
    df.dropna(how='any', inplace=True)
    
    # #Exporting as csv file to program folder
    df.to_csv("{}_posts.csv".format(search), encoding='utf-8', index=False)
    
    return df

In [ ]:
def word_cloud():
    # Python program to generate WordCloud

    # importing all necessary modules
    from wordcloud import WordCloud, STOPWORDS
    import matplotlib.pyplot as plt
    import pandas as pd

    words = ''
    stopwords = set(stopwords.words("english"))
    stopwords.update(["zomato", "india", "food", "business"])

    # iterate through the csv file
    for val in df.iloc[:,1]:

        # typecaste each val to string
        val = str(val)

        # split the value
        tokens = val.split()

        # Converts each token into lowercase
        for i in range(len(tokens)):
            tokens[i] = tokens[i].lower()

        words += " ".join(tokens)+" "

    wordcloud = WordCloud(width = 800, height = 800,
                    background_color ='white',
                    stopwords = stopwords,
                    min_font_size = 10).generate(words)

    # plot the WordCloud image
    plt.figure(figsize = (8, 8), facecolor = None)
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.tight_layout(pad = 0)

    plt.show()


In [ ]:
def analysis_model():
    import numpy as np # linear algebra
    import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
    import os
    import matplotlib.pyplot as plt
    import seaborn as snb
    import string
    
    